## Install some packages

In [2]:
%pip install --upgrade pip 
%pip install --disable-pip-version-check torch==1.13.1 torchdata==0.5.1 --quiet
%pip install transformers==4.27.2 datasets==2.11.0 --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement torch==1.13.1 (from versions: 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1)
ERROR: No matching distribution found for torch==1.13.1


Note: you may need to restart the kernel to use updated packages.


## import some libraries 

In [1]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig

## Load Dataset

In [2]:
huggingface_dataset_name= "knkarthick/dialogsum"
dataset= load_dataset(huggingface_dataset_name)

Found cached dataset csv (C:/Users/DELL/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-cd36827d3490488d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/3 [00:00<?, ?it/s]

## Explore this dataset 

In [32]:
example_indices=[1,55]
dash_line ='_'.join("" for x in range(100))

for i , index in enumerate(example_indices):
    print(dash_line)
    print('Example ', i+1)
    print(dash_line)
    print("INPUT DIALOGUE: ")
    print(dataset['test'] [ index] ['dialogue'])
    print(dash_line)
    print("BASELINE HUMAN SUMMARY:")
    print(dataset['test'][index]['summary'])
    print(dash_line)


___________________________________________________________________________________________________
Example  1
___________________________________________________________________________________________________
INPUT DIALOGUE: 
#Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
#Person2#: Yes, sir. Go ahead.
#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.
#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?
#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.
#Person2#: But sir, many employees use Instant Messaging to 

## load the pretrained model & Tokenizer



Here are some pretrained models that we can use and there cons

BART (Bidirectional and Auto-Regressive Transformers):

Model: facebook/bart-large-cnn
Pros: Strong performance on various summarization benchmarks, handles long sequences well, pre-trained on diverse datasets.
Use Case: General-purpose summarization, including dialogues.
T5 (Text-To-Text Transfer Transformer):

Model: t5-large or t5-base
Pros: Flexible text-to-text framework, strong performance across multiple NLP tasks including summarization.
Use Case: Summarization, translation, question answering, and other text generation tasks.
Pegasus (Pre-training with Extracted Gap-sentences for Abstractive Summarization):

Model: google/pegasus-large
Pros: Designed specifically for summarization, excellent performance on summarization tasks, trained on a large corpus.
Use Case: High-quality abstractive summarization.

we're gonna use flan-t5

In [4]:
model_name= "google/flan-t5-base"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [5]:
tokenizer=AutoTokenizer.from_pretrained (model_name,use_fast=True)

## Tokenize & Embading ====> Encoding &  vice versa ====> Decoding

In [26]:
for i, index in enumerate( example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    inputs= tokenizer(dialogue,return_tensors='pt') # it returns a pytorch tensor
    output= tokenizer.decode(model.generate(
        inputs['input_ids'], # input_ids ==> is a part from the input tensor consist from token vectors 
        max_new_tokens=50

    )[0],
    skip_special_tokens=True)
    print(dash_line)
    print("Example",i+1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY: \n {summary}')
    print(dash_line)
    print(f'MODEL GENERATION WITHOUT PROMPT ENGINEERING:\n{output}')


___________________________________________________________________________________________________
Example 1
___________________________________________________________________________________________________
INPUT PROMPT:
#Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
#Person2#: Yes, sir. Go ahead.
#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.
#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?
#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.
#Person2#: But sir, many employees use Instant Messaging to comm

## IN-CONTEXT LEARNING 

### Zero shot With prompt

In [27]:
for i, index in enumerate( example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt= f"""Dialogue

    {dialogue}
    
    Summary: 
    """

    inputs= tokenizer(prompt,return_tensors='pt') # it returns a pytorch tensor
    output= tokenizer.decode(model.generate(
        inputs['input_ids'], # input_ids ==> is a part from the input tensor consist from token vectors 
        max_new_tokens=50

    )[0],
    skip_special_tokens=True)
    print(dash_line)
    print("Example",i+1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY: \n {summary}')
    print(dash_line)
    print(f'MODEL GENERATION WITH ZERO PROMPTED SHOT :\n{output}')


___________________________________________________________________________________________________
Example 1
___________________________________________________________________________________________________
INPUT PROMPT:
#Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
#Person2#: Yes, sir. Go ahead.
#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.
#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?
#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.
#Person2#: But sir, many employees use Instant Messaging to comm

### Make prompt 

In [28]:
def make_prompt (example_indices,example_indices_to_summarize ):
    prompt = ""
    if isinstance(example_indices, ( float, str)):
        print( "not accepted ==> must me integer of list of integers") 
    else:
        if  isinstance(example_indices, (int)):
            example_indices= [example_indices]
        else :
            example_indices=example_indices
        for j in example_indices:
            dialogue= dataset['test'][j]['dialogue']
            summary = dataset['test'][j]['summary']
            prompt += f"""Dialouge: 

            {dialogue}

            summary: 

            {summary}

            """   
        prompt +=f"""Dialogue: 

        {dataset['test'][example_indices_to_summarize]['dialogue']}

        Summary: 
        
        """
    return prompt


### One Shot - ONE COMPLETE EXAMPLE - 

In [29]:
for i, index in enumerate( example_indices):
    summary = dataset['test'][index]['summary']

    prompt= make_prompt(34,index)



    inputs= tokenizer(prompt,return_tensors='pt') # it returns a pytorch tensor
    output= tokenizer.decode(model.generate(
        inputs['input_ids'], # input_ids ==> is a part from the input tensor consist from token vectors 
        max_new_tokens=50

    )[0],
    skip_special_tokens=True)
    print(dash_line)
    print("Example",i+1)
    print(dash_line)
    print(f"BASELINE HUMAN SUMMARY: \n {summary}")
    print(dash_line)
    print(f'MODEL GENERATION WITH ZERO SHOT :\n{output}')


___________________________________________________________________________________________________
Example 1
___________________________________________________________________________________________________
BASELINE HUMAN SUMMARY: 
 In order to prevent employees from wasting time on Instant Message programs, #Person1# decides to terminate the use of those programs and asks Ms. Dawson to send out a memo to all employees by the afternoon.
___________________________________________________________________________________________________
MODEL GENERATION WITH ZERO SHOT :
The memo is to be distributed to all employees by this afternoon.
___________________________________________________________________________________________________
Example 2
___________________________________________________________________________________________________
BASELINE HUMAN SUMMARY: 
 #Person1# wants to refurbish the kitchen while #Person2# thinks it unnecessary.
________________________________________

### Few Shots 

In [36]:
example_indice=[34,56,11,2,4]

### One Shot - ONE COMPLETE EXAMPLE - 
for i, index in enumerate( example_indices):
    summary = dataset['test'][index]['summary']

    prompt= make_prompt(example_indice,index)



    inputs= tokenizer(prompt,return_tensors='pt') # it returns a pytorch tensor
    output= tokenizer.decode(model.generate(
        inputs['input_ids'], # input_ids ==> is a part from the input tensor consist from token vectors 
        max_new_tokens=50

    )[0],
    skip_special_tokens=True)
    print(dash_line)
    print("Example",i+1)
    print(dash_line)
    print(f"BASELINE HUMAN SUMMARY: \n {summary}")
    print(dash_line)
    print(f'MODEL GENERATION WITH FEW SHOTS SHOT :\n{output}')


___________________________________________________________________________________________________
Example 1
___________________________________________________________________________________________________
BASELINE HUMAN SUMMARY: 
 In order to prevent employees from wasting time on Instant Message programs, #Person1# decides to terminate the use of those programs and asks Ms. Dawson to send out a memo to all employees by the afternoon.
___________________________________________________________________________________________________
MODEL GENERATION WITH FEW SHOTS SHOT :
Ms. Dawson takes a dictation for #Person1# about prohibiting the use of Instant Message programs in the office.
___________________________________________________________________________________________________
Example 2
___________________________________________________________________________________________________
BASELINE HUMAN SUMMARY: 
 #Person2# finds that #Person1# has a fever and says #Person1# should'

## Configration parameters

In [39]:
generation_config= GenerationConfig(max_new_tokens= 50,do_sample=True,top_k =3,top_p=.4,temperature=2.00)

#generation_config= GenerationConfig(max_new_tokens= 50,do_sample=True,top_k =6,top_p=.3,temperature=.1)
#generation_config= GenerationConfig(max_new_tokens= 50,do_sample=True,top_k =3,top_p=.5,temperature=1)
#generation_config= GenerationConfig(max_new_tokens= 50,do_sample=True,top_k =3,top_p=.4,temperature=.5)


example_indice=[34,56,11,2,4]

for i, index in enumerate( example_indices):
    summary = dataset['test'][index]['summary']

    prompt= make_prompt(example_indice,index)



    inputs= tokenizer(prompt,return_tensors='pt') # it returns a pytorch tensor
    output= tokenizer.decode(model.generate(
        inputs['input_ids'], # input_ids ==> is a part from the input tensor consist from token vectors 
        generation_config=generation_config

    )[0],
    skip_special_tokens=True)
    print(dash_line)
    print("Example",i+1)
    print(dash_line)
    print(f"BASELINE HUMAN SUMMARY: \n {summary}")
    print(dash_line)
    print(f'MODEL GENERATION WITH FEW SHOTS SHOT :\n{output}')



___________________________________________________________________________________________________
Example 1
___________________________________________________________________________________________________
BASELINE HUMAN SUMMARY: 
 In order to prevent employees from wasting time on Instant Message programs, #Person1# decides to terminate the use of those programs and asks Ms. Dawson to send out a memo to all employees by the afternoon.
___________________________________________________________________________________________________
MODEL GENERATION WITH FEW SHOTS SHOT :
Ms. Dawson takes a dictation for #Person1# about prohibiting the use of Instant Message programs in the office.
___________________________________________________________________________________________________
Example 2
___________________________________________________________________________________________________
BASELINE HUMAN SUMMARY: 
 #Person2# finds that #Person1# has a fever and says #Person1# should'